# Initial Data Exploration

This notebook does some early cleaning of a data frame with indicators for child well-being in OECD member nations and those same indicators for some nations who are not in the OECD. This notebook also gives a general look at the distribution of some of the variables. The data was pulled for the [OECD's database](https://stats.oecd.org/) underneath the section on "Social Protection and Well-Being" which also includes data on poverty/wealth distribution, gender, and general well-being. I wanted any data I used to be recent and I wanted to do some temporal analysis, so I chose data from 2014 to 2018 to explore. These explorations include: 
* How many observations and variables are there?
* How many data values are there for each year in the 5-year period?
* How many data values are there for each indicator of child-well-being?
* For each country in the data set, which year provides the most data values? 
* For each indicator, how many values are there per year?

---------

#### HISTORY

* 10/25/20 aheyward - Initial exploration of OECD dataset

In [1]:
#Importing necessary packages
import pandas as pd

In [2]:
#Importing the raw data from the data folder
oecd_df = pd.read_csv("../data/raw/OECD_child_wellbeing.csv")

#### How many observations and variables are there?
By pulling the shape of the data, we can see how many observation it has in its raw state.

In [3]:
#Getting the shape of the DataFrame rem_df and use two pointers 
ncol = oecd_df.shape[1] #indexing the number of columns
nrow = oecd_df.shape[0] #indexing the number of rows

print(ncol)
print(nrow)

9
2335


There are 2,153 observations in this data set, but there are only 37 OECD member countries, in addition to data from 10 non-OECD nations. Where do all these extra observations come from? I will run `head()` to gauge the unit of analysis. 

In [4]:
oecd_df.head(10)

,COU,Country,IND,Indicator,YEAR,Year,Value,Flag Codes,Flags
0,AUS,Australia,CWB3,"Average disposable household income, 0-17 year...",2014,2014,34872.4,NaN,NaN
1,AUS,Australia,CWB3,"Average disposable household income, 0-17 year...",2016,2016,34788.7,NaN,NaN
2,AUS,Australia,CWB8A,Adolescents (15-year-olds) who report talking ...,2015,2015,96.7,NaN,NaN
3,AUS,Australia,CWB16,Adolescents (15-year-olds) skipping either bre...,2015,2015,22.1,NaN,NaN
4,AUS,Australia,CWB17A,Adolescents (15-year-olds) who report doing no...,2015,2015,6.5,NaN,NaN
5,AUS,Australia,CWB17B,Adolescents (15-year-olds) who report regularl...,2015,2015,52.5,NaN,NaN
6,AUS,Australia,CWB22,Children (3-5 year-olds) enrolled in pre-prima...,2014,2014,85.3,NaN,NaN
7,AUS,Australia,CWB22,Children (3-5 year-olds) enrolled in pre-prima...,2015,2015,86.5,NaN,NaN
8,AUS,Australia,CWB22,Children (3-5 year-olds) enrolled in pre-prima...,2016,2016,84.7,NaN,NaN
9,AUS,Australia,CWB22,Children (3-5 year-olds) enrolled in pre-prima...,2017,2017,84.0,NaN,NaN


In [5]:
oecd_df.columns

Index(['COU', 'Country', 'IND', 'Indicator', 'YEAR', 'Year', 'Value',
       'Flag Codes', 'Flags'],
      dtype='object')

#### How many values are there in each column?
Here you can see that each row is the data value for one indicator of child well-being for a specific country in a given year. How many countries, years, and indicators are included int the data?

In [6]:
oecd_df.nunique()

COU            50
Country        50
IND            29
Indicator      29
YEAR            5
Year            5
Value         975
Flag Codes      0
Flags           0
dtype: int64

There appear to be 50 countries in the data, data from 5 different years, and 29 indicators. Despite there being 2,335 observations, there are only 975 actual data values, which means there must be a great deal of missing data. There are also two variables for which no data is recorded at all. Which year had the data recorded for the national indicators?

In [7]:
oecd_df['Year'].value_counts()

2015    1048
2014     395
2016     341
2017     313
2018     238
Name: Year, dtype: int64

2015 has the most data out of the 5 years. Breaking it down further below by country, you can see that not all countries have data for a certain indicator, and when they do, it is not consistent across each year. For example, below you can see that Australia has values for 4 indicators in 2014, while the US 9 for the same year. However, the US also has a whopping 24 values recorded for the following year, 2015. 

In [8]:
oecd_df.groupby('Country')['Year'].value_counts()

Country        Year
Australia      2015    17
               2014     4
               2016     4
               2017     3
               2018     3
                       ..
United States  2015    24
               2014     9
               2016     9
               2017     9
               2018     8
Name: Year, Length: 235, dtype: int64

### Cleaning the data frame
Here, we renamed the columns so they make more sense and have a uniform format.
Then, we subsetted the original data frame to get rid of columns that were nothing but missing values for each observation and applied the new column names.

In [9]:
#Creating a list that has only the columns going into the new data frame.
cols_to_use = ['COU', 'Country', 'IND', 'Indicator', 'YEAR', 'Value']

#Creating a dictionary with new column names (values) that correspond with the original column names (keys)
col_map = {'COU': 'country_code',
           'Country': 'country',
           'IND': 'indicator_code',
           'Indicator': 'indicator',
           'YEAR': 'year',
           'Value': 'data_value'}

#Filtering the original data frame by the list of columns and applying the dictionary values
oecd_df2 = oecd_df[cols_to_use].rename(columns=col_map)

In [10]:
#Checkin the new layout of the data frame
oecd_df2.head(5)

,country_code,country,indicator_code,indicator,year,data_value
0,AUS,Australia,CWB3,"Average disposable household income, 0-17 year...",2014,34872.4
1,AUS,Australia,CWB3,"Average disposable household income, 0-17 year...",2016,34788.7
2,AUS,Australia,CWB8A,Adolescents (15-year-olds) who report talking ...,2015,96.7
3,AUS,Australia,CWB16,Adolescents (15-year-olds) skipping either bre...,2015,22.1
4,AUS,Australia,CWB17A,Adolescents (15-year-olds) who report doing no...,2015,6.5


In [11]:
#Saving the data frame as a .csv file in the `cleaned` data folder
oecd_df2.to_csv('../data/cleaned/CLEAN_OECD_child_wellbeing.csv', index=False)

Next, it would be good to get a sense of how many indicators are present for each country in a given year.

In [12]:
#A grouped chart to see differences in data across time.
indicators_by_loc_and_year = oecd_df2.groupby(['country_code', 'year'])['indicator_code'].count().unstack()
indicators_by_loc_and_year

year,2014,2015,2016,2017,2018
country_code,,,,,
AUS,4.0,17.0,4.0,3.0,3.0
AUT,12.0,26.0,10.0,9.0,8.0
BEL,9.0,26.0,10.0,9.0,8.0
BEL-VLG,NaN,1.0,NaN,NaN,NaN
BGR,9.0,23.0,7.0,7.0,6.0
BRA,1.0,17.0,1.0,1.0,1.0
CAN,5.0,16.0,6.0,3.0,2.0
CHE,10.0,23.0,7.0,6.0,3.0
CHL,3.0,21.0,3.0,4.0,1.0


In [13]:
#A bar graph to quickly visualize which year has the most data available.
indicators_by_loc_and_year.sort_index(ascending=False).plot(kind='barh',figsize=(6,20))

<AxesSubplot:ylabel='country_code'>

Given the results of the bar graph, it again appears that 2015 has the most data available by far. There is very little data available for any year for non-OCED  nations.
* Indonesia does not have many indicators for any year, and Peru only has data for indicators in 2015. South Africa and Flanders also only have data for 2015, but there are less than 5 values recorded for either.
* For each country, all of the other years have fewer than 15 data values, suggesting that the other four years do not offer very much data for substantial analysis.

#### How many values are there per data indicator?

In [14]:
oecd_df2.groupby(['indicator_code', 'indicator']).size().sort_values(ascending=False).to_frame()

,,0
indicator_code,indicator,
CWB50B,Length of paid paternity and parental leave reserved for fathers in weeks,181
CWB50A,Length of paid maternity and parental leave available to mothers in weeks,181
CWB22,Children (3-5 year-olds) enrolled in pre-primary education or primary school (%),174
CWB1A,Children (0-17) living with two parents (%),157
CWB1B,Children (0-17) living with a single parent (%),157
CWB1C,Children (0-17) living in 'other' types of household (%),157
CWB2A,Children (0-14) in households where all adults are in employment (working) (%),149
CWB2C,Children (0-14) in households with all adults not in employment (jobless) (%),149
CWB3,"Average disposable household income, 0-17 year-olds, 2015 USD PPP",114


The indicators that look most promising because they have the most data are length of maternity leave, info about primary school, information about parental employment, and indicators about familial structure. All of these have well over 100 values recorded.

#### For each indicator, how many values are there per year?

In [15]:
oecd_df2.groupby(['indicator_code', 'indicator','year']).size().sort_values(ascending=False).unstack()

,year,2014,2015,2016,2017,2018
indicator_code,indicator,,,,,
CWB16,Adolescents (15-year-olds) skipping either breakfast or dinner (%),NaN,44.0,NaN,NaN,NaN
CWB17A,Adolescents (15-year-olds) who report doing no physical activity outside of school (%),NaN,43.0,NaN,NaN,NaN
CWB17B,Adolescents (15-year-olds) who report regularly engaging in vigorous physical activity outside of school (%),NaN,43.0,NaN,NaN,NaN
CWB1A,Children (0-17) living with two parents (%),33.0,34.0,34.0,32.0,24.0
CWB1B,Children (0-17) living with a single parent (%),33.0,34.0,34.0,32.0,24.0
CWB1C,Children (0-17) living in 'other' types of household (%),33.0,34.0,34.0,32.0,24.0
CWB22,Children (3-5 year-olds) enrolled in pre-primary education or primary school (%),40.0,45.0,44.0,45.0,NaN
CWB27,Adolescents (15-year-olds) with parents interested in and supportive of their education (%),NaN,43.0,NaN,NaN,NaN
CWB28,Adolescents (15-year-olds) who study before or after school (%),NaN,44.0,NaN,NaN,NaN


Examining this further by year, the reason why the aforementioned variables have so many values recorded is because they are the only ones where information was recorded across all years. 